# Chapter 6 - Complex Structure - JSON
Create JSON in a dataframe column.

# Setup

In [2]:
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row

import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.DataFrame

import java.time.temporal.ChronoUnit
import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp

### Spark parition control based on core availability

In [3]:
val NUM_CORES = 2
val NUM_PARTITIONS = 2

lazy val spark: SparkSession = SparkSession.builder()
    .appName("dataframe-json")
    .getOrCreate()

spark.conf.set("spark.default.parallelism", 8)
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
/*
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.driver.memory", "6g")
spark.conf.set("spark.executor.memory", "2g")
spark.conf.set("spark.master", "spark://masa:7077")
*/
import spark.implicits._

NUM_CORES = 2
NUM_PARTITIONS = 2
spark = <lazy>


<lazy>

In [4]:
val configMap = spark.conf.getAll.foreach(println)

(spark.serializer,org.apache.spark.serializer.KryoSerializer)
(spark.driver.host,192.168.1.116)
(spark.eventLog.enabled,true)
(spark.driver.port,43377)
(spark.hadoop.validateOutputSpecs,True)
(spark.repl.class.uri,spark://192.168.1.116:43377/classes)
(spark.jars,file:/home/oonisim/.local/share/jupyter/kernels/apache_toree_scala/lib/toree-assembly-0.3.0-incubating.jar)
(spark.repl.class.outputDir,/tmp/spark-ee7db58f-554b-46c0-b5d8-3be1b666c151/repl-6d42b196-e318-4058-a217-5e26e818ce1b)
(spark.app.name,flight)
(spark.driver.memory,2g)
(spark.executor.instances,2)
(spark.history.fs.logdirectory,hdfs://localhost:8020/logs_spark)
(spark.default.parallelism,8)
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.master,local)
(spark.executor.memory,4g)
(spark.eventLog.dir,hdfs://localhost:8020/logs_spark)
(spark.executor.cores,4)
(spark.app.id,local-1575884632753)
(spark.sql.shuffle.partitions,4)


configMap: Unit = ()


# Dataframe

In [31]:
val jsonDF = spark.range(1).selectExpr("""
  '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString
""")
jsonDF.show(false)

+-------------------------------------------+
|jsonString                                 |
+-------------------------------------------+
|{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}|
+-------------------------------------------+



jsonDF = [jsonString: string]


[jsonString: string]

## get_json_object

In [36]:
jsonDF.select(
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[2]")
).show(false)

+-------------------------------------------------------+
|get_json_object(jsonString, $.myJSONKey.myJSONValue[2])|
+-------------------------------------------------------+
|3                                                      |
+-------------------------------------------------------+



## to_Json

In [39]:
df.selectExpr(
    "to_json((InvoiceNo, Description)) as myStruct"
)
.select(col("myStruct"))
.show(3, false)

+-------------------------------------------------------------------------+
|myStruct                                                                 |
+-------------------------------------------------------------------------+
|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}|
|{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}               |
|{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}    |
+-------------------------------------------------------------------------+
only showing top 3 rows



In [42]:
df.selectExpr(
    "to_json((InvoiceNo, Description)) as myStruct"
)
.select(
    get_json_object(col("myStruct"), "$.Description")
).show(3, false)

+----------------------------------------+
|get_json_object(myStruct, $.Description)|
+----------------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER      |
|WHITE METAL LANTERN                     |
|CREAM CUPID HEARTS COAT HANGER          |
+----------------------------------------+
only showing top 3 rows

